***GENERATED CODE FOR classtest PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PredcitiveChurnTrain[State]', 'transformation_label': 'String Indexer'}], 'feature': 'PredcitiveChurnTrain[State]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '1', 'mean': '', 'stddev': '', 'min': 'MI', 'max': 'MI', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PredcitiveChurnTrain[Stat...'}, {'feature_label': 'PredcitiveChurnTrain[State]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'PredcitiveChurnTrain[State]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PredcitiveChurnTrain[Phone]', 'transformation_label': 'String Indexer'}], 'feature': 'PredcitiveChurnTrain[Phone]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1', 'mean': '', 'stddev': '', 'min': '363-5450', 'max': '363-5450', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PredcitiveChurnTrain[Phon...'}, {'feature_label': 'PredcitiveChurnTrain[Phone]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'PredcitiveChurnTrain[Phone]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PredcitiveChurnTrain[cluster_labels]', 'transformation_label': 'String Indexer'}], 'feature': 'PredcitiveChurnTrain[cluster_labels]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'day_callers', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PredcitiveChurnTrain[clus...'}, {'feature_label': 'PredcitiveChurnTrain[cluster_labels]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'PredcitiveChurnTrain[cluster_labels]')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run classtestHooks.ipynb
try:
	#sourcePreExecutionHook()

	classtesthdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1636107854.csv', 'filename': 'tempFile1636107854.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(classtesthdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classtestHooks.ipynb
try:
	#transformationPreExecutionHook()

	classtestautofe = TransformationMain.run(classtesthdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "PredcitiveChurnTrain[State]", "transformation_label": "String Indexer"}], "feature": "PredcitiveChurnTrain[State]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "MI", "max": "MI", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Stat..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Account_Length]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "65.0", "stddev": "nan", "min": "65", "max": "65", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Acco..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Area_Code]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "415.0", "stddev": "nan", "min": "415", "max": "415", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Area..."}, {"transformationsData": [{"feature_label": "PredcitiveChurnTrain[Phone]", "transformation_label": "String Indexer"}], "feature": "PredcitiveChurnTrain[Phone]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "363-5450", "max": "363-5450", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Phon..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Intl_Plan]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Intl..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[VMail_Plan]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[VMai..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[VMail_Message]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[VMai..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Day_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "211.3", "stddev": "nan", "min": "211.3", "max": "211.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Day_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Day_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "120.0", "stddev": "nan", "min": "120", "max": "120", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Day_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Day_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "35.92", "stddev": "nan", "min": "35.92", "max": "35.92", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Day_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Eve_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "162.6", "stddev": "nan", "min": "162.6", "max": "162.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Eve_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Eve_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "122.0", "stddev": "nan", "min": "122", "max": "122", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Eve_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Eve_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "13.82", "stddev": "nan", "min": "13.82", "max": "13.82", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Eve_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Night_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "134.7", "stddev": "nan", "min": "134.7", "max": "134.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Nigh..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Night_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "118.0", "stddev": "nan", "min": "118", "max": "118", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Nigh..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Night_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "6.06", "stddev": "nan", "min": "6.06", "max": "6.06", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Nigh..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Intl_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "13.2", "stddev": "nan", "min": "13.2", "max": "13.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Intl..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[total_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "521.8", "stddev": "nan", "min": "521.8", "max": "521.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[tota..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Intl_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "5.0", "stddev": "nan", "min": "5", "max": "5", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Intl..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Intl_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "3.56", "stddev": "nan", "min": "3.56", "max": "3.56", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Intl..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Total_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "59.36", "stddev": "nan", "min": "59.36", "max": "59.36", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[Tota..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[CustServ_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "3.0", "stddev": "nan", "min": "3", "max": "3", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Cust..."}, {"transformationsData": [{"feature_label": "PredcitiveChurnTrain[cluster_labels]", "transformation_label": "String Indexer"}], "feature": "PredcitiveChurnTrain[cluster_labels]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "day_callers", "max": "day_callers", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PredcitiveChurnTrain[clus..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredcitiveChurnTrain[Churn]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PredcitiveChurnTrain[Chur..."}]}))

	#transformationPostExecutionHook(classtestautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classtestHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classtestautofe, ["PredcitiveChurnTrain[Account_Length]", "PredcitiveChurnTrain[Area_Code]", "PredcitiveChurnTrain[Intl_Plan]", "PredcitiveChurnTrain[VMail_Plan]", "PredcitiveChurnTrain[VMail_Message]", "PredcitiveChurnTrain[Day_Mins]", "PredcitiveChurnTrain[Day_Calls]", "PredcitiveChurnTrain[Day_Charge]", "PredcitiveChurnTrain[Eve_Mins]", "PredcitiveChurnTrain[Eve_Calls]", "PredcitiveChurnTrain[Eve_Charge]", "PredcitiveChurnTrain[Night_Mins]", "PredcitiveChurnTrain[Night_Calls]", "PredcitiveChurnTrain[Night_Charge]", "PredcitiveChurnTrain[Intl_Mins]", "PredcitiveChurnTrain[total_Mins]", "PredcitiveChurnTrain[Intl_Calls]", "PredcitiveChurnTrain[Intl_Charge]", "PredcitiveChurnTrain[Total_Charge]", "PredcitiveChurnTrain[CustServ_Calls]", "PredcitiveChurnTrain[State]_stringindexer", "PredcitiveChurnTrain[Phone]_stringindexer", "PredcitiveChurnTrain[cluster_labels]_stringindexer"], "PredcitiveChurnTrain[Churn]")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

